# Real-Time Plotting of Arduino Data

**UNDER DEVELOPMENT!**

Based on YT video<br>
_Create GUI apps in Jupyter using ipywidgets_ by **Everton Colling**:<br>
https://www.youtube.com/watch?v=f0WmLo8AVxo

Other inspirations:
* https://stackoverflow.com/questions/70625801/threading-reading-a-serial-port-in-python-with-a-gui

In [1]:
# import libraries
from ipywidgets.widgets import Label, FloatProgress, FloatSlider, Button, Dropdown
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display
import bqplot as bq
import numpy as np
import time
import threading
import serial, serial.tools.list_ports

In [2]:
BUFSIZE = 1200
x = np.zeros(BUFSIZE)
y = np.zeros(BUFSIZE)

In [3]:
flag = True

In [4]:
list_ports = serial.tools.list_ports.comports()
portnames = [port.name for port in list_ports]

In [5]:
# Dropdown with all known ports
dd_ports = Dropdown(
    options=portnames,
    value=portnames[-1],
    description='Port:',
    disabled=False
)

def ports_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

dd_ports.observe(ports_on_change)

# display(dd_ports)

In [6]:
# Button "Open Port"
b_open = Button(
    description = "Open Port", 
    icon="start",
    button_style="warning",
    layout = Layout(width="100px")
)

def open_click(b):
    open_ser_port(dd_ports.value)
    
b_open.on_click(open_click)

def open_ser_port(portname):
    try:
        global ser
        ser = serial.Serial()
        BAUDRATE = 115200
        ser.baudrate = BAUDRATE
        ser.port = portname
        ser.open()
    except:
        print(f"Error! Could not open port {portname:s}.")
        
    print(f"Port {portname:s} is open: {bool(ser.is_open)}.")
    
# display(b_open)

In [7]:
# Open port dialog
port_box = HBox(
    children=(dd_ports, b_open),
    layout=Layout(border="solid 2px gray", width="450px")
)

display(port_box)

In [9]:
b_stop = Button(
    description = "Stop", 
    icon="stop",
    button_style="warning",
    layout = Layout(width="100px")
)

def stop_click(b):
    global flag
    flag = False
    
b_stop.on_click(stop_click)

In [10]:
# Plot elements
x_sc=bq.LinearScale()
y_sc=bq.LinearScale()

x_ax=bq.Axis(
    label="x(t)",
    scale=x_sc
)

y_ax=bq.Axis(
    label="y(t)",
    scale=y_sc,
    orientation = "vertical"
)

Line = bq.Lines(
    x=x,
    y=y,
    scales={"x": x_sc, "y": y_sc}
)

fig=bq.Figure(
    layout=Layout(width="800px", height="400px"),
    axes=[x_ax, y_ax],
    marks=[Line],
    fig_margin=dict(top=10, bottom=40, left=50, right=10)
)

#fig

In [11]:
app = VBox(
    children=(b_stop, fig)
)

In [12]:
# Loop function
def work():
    global x
    global y
    
    # Our standard Arduino program charges a cap during 6 secs and discharges it during 6 secs.
    # The sampling period is T=20ms, equivalent to a sampling freq. of f = 1/20ms = 50 samples / sec.
    # Our Arduino software reports 4 data columns:
    # ms: milliseconds since the last Arduino reset
    # V0: charging/discharging voltage
    # VC: capacitor voltage
    # Vd: digital input state, 0mV or 5000mV
    
    T = 20. # ms
    
    x = np.zeros(BUFSIZE)
    y = np.zeros(BUFSIZE)


    ser.flush()

    Line.x = x
    Line.y = y

    k=0
    
    while flag:
        line = ser.readline()
        if line: 
            try:
                lst = [int(s) for s in line.split()]

                ms, V0, VC, Vd = lst
                
                if k==0: # first call
                    x = ms + T*np.arange(BUFSIZE)
                
                if k<BUFSIZE:
                    x[k]=ms
                    y[k]=VC
               
                else:
                    x = np.delete(x,0)
                    y = np.delete(y,0)

                    x = np.append(x,ms)
                    y = np.append(y,VC)

                k += 1

                Line.x = x
                Line.y = y
                
               
            except:
                ser.flush()
                pass

        time.sleep(0.01)

In [13]:
# GOGOGO!!!

flag = True

thread = threading.Thread(target=work)

display(app)

thread.start()

In [ ]:
thread.is_alive()

In [ ]:
x